In [2]:
import pandas as pd
import numpy as np

# Load the national ZIP code dataset
zip_to_county_df = pd.read_csv('../data/raw_data/zip.csv')

# Filter to include only Jefferson County, KY codes and create a copy
jefferson_zip_df = zip_to_county_df[
    (zip_to_county_df['county'] == 'Jefferson County') & 
    (zip_to_county_df['state'] == 'KY')
].copy()

# Convert 'zip' column to numeric, handling errors, and cast to nullable integer
jefferson_zip_df['zip'] = pd.to_numeric(jefferson_zip_df['zip'], errors='coerce').astype('Int64')

# Keep only relevant columns
jefferson_zip_df = jefferson_zip_df[['zip']]

# Reset the index
jefferson_zip_df.reset_index(drop=True, inplace=True)

# Save the resulting DataFrame
jefferson_zip_df.to_csv('../data/jefferson_zip_df.csv', index=False)

# Display the DataFrame
jefferson_zip_df


,zip
0,40018
1,40023
2,40025
3,40027
4,40041
...,...
68,40295
69,40296
70,40297
71,40298


In [3]:

import pandas as pd

# Define the file paths
file_paths = ['../data/raw_data/2020.csv', 
              '../data/raw_data/2021.csv', 
              '../data/raw_data/2022.csv', 
              '../data/raw_data/2023.csv', 
              '../data/raw_data/2024.csv']

columns_to_drop = ['block_address', 'BLOCK_ADDRESS', 'city', 'City','badge_id', 'BADGE_ID', 'ObjectId', 'nibrs_code', 'NIBRS_CODE','nibrs_group_name','UCR_HIERARCHY', 'LMPD_BEAT', 'lmpd_beat', 'LMPD_DIVISION', 'lmpd_division',]

# Standardize column names with dictionary
rename_columns = {
    'ZIP_CODE' : 'zip',
    'zip_code': 'zip',
    'INCIDENT_NUMBER' : 'incident_number',
    'DATE_REPORTED': 'date_reported',
    'DATE_OCCURED': 'date_occurred',
    'CRIME_TYPE': 'offense_classification',
    'PREMISE_TYPE' : 'location_category',
    'ATT_COMP' : 'was_offense_completed',
    'UOR_DESC' : 'offense_code_name'
}

# Step 3: Read and combine the files
data_frames = []

for file in file_paths:
    df = pd.read_csv(file, low_memory=False)
    
    df.drop(columns=columns_to_drop, errors='ignore', inplace=True)
    
    df.rename(columns=rename_columns, inplace=True)

    df = df[df['offense_classification'].isin(['MOTOR VEHICLE THEFT', '14 AUTO THEFT'])].copy()
    df['offense_classification'] = 'AUTO THEFT'
    
    df['was_offense_completed'] = df['was_offense_completed'].replace(
        {'COMPLETED': 'YES', 'ATTEMPTED': 'NO'}, regex=True
    ).fillna('UNKNOWN')
    
    df['location_category'] = df['location_category'].str.strip()
    # df['offense_code_name'] = df['offense_code_name'].str.extract(
    # r'(\$[\d,]+(?: < \$[\d,]+|,000,000(?: OR MORE)?))')
    # df['offense_code_name'] = df['offense_code_name'].fillna('UNKNOWN VALUE')
    df['zip'] = (
    df['zip']
    .astype(str)
    .str.strip()
    .str.split('-').str[0]
    .str.replace(r'\.0$', '', regex=True)
    .replace(['99999', 'nan', None, '', 'NaN'], pd.NA)
)

# Keep valid ZIP codes
    df = df[df['zip'].str.match(r'^\d{5}$', na=False)]
    df['zip'] = pd.to_numeric(df['zip'], errors='coerce').astype('Int64')
    data_frames.append(df)

# Combine all DataFrames into one
crime_df = pd.concat(data_frames, ignore_index=True)

# # Perform a left join on 'zip'
jefferson_crime_df = jefferson_zip_df.merge(crime_df, how='inner', on='zip')
jefferson_crime_df['date_reported'] = jefferson_crime_df['date_reported'].str.strip()
jefferson_crime_df['date_reported'] = jefferson_crime_df['date_reported'].str.replace('/', '-')
jefferson_crime_df['date_occurred'] = jefferson_crime_df['date_occurred'].str.strip()
jefferson_crime_df['date_occurred'] = jefferson_crime_df['date_occurred'].str.replace('/', '-')
jefferson_crime_df['date_reported'] = pd.to_datetime(jefferson_crime_df['date_reported'], format='mixed', utc=True).dt.tz_localize(None)
jefferson_crime_df['date_occurred'] = pd.to_datetime(jefferson_crime_df['date_occurred'], format='mixed', utc=True).dt.tz_localize(None)

regex_pattern = (
    r"(< \$500|"                      # Matches "< $500"
    r"\$500 < \$1,000|"               # Matches "$500 < $1,000"
    r"\$500 < \$10,000|"              # Matches "$500 < $10,000"
    r"\$1,000 < \$10,000|"            # Matches "$1,000 < $10,000"
    r"\$10,000 < \$1,000,000|"        # Matches "$10,000 < $1,000,000"
    r"\$1,000,000 < \$10,000,000|"    # Matches "$1,000,000 < $10,000,000"
    r"\$10,000,000 OR MORE|"          # Matches "$10,000,000 OR MORE"
    r"> \$500 BUT < \$10,000|"        # Matches "> $500 BUT < $10,000"
    r"> \$10,000 BUT < \$1,000,000)"  # Matches "> $10,000 BUT < $1,000,000"
)

# Extract monetary ranges using the regex pattern
jefferson_crime_df['value_range'] = jefferson_crime_df['offense_code_name'].str.extract(regex_pattern, expand=False)

# Normalize inconsistent ranges
jefferson_crime_df['value_range'] = jefferson_crime_df['value_range'].replace({
    "> $500 BUT < $10,000": "$500 < $10,000",
    "> $10,000 BUT < $1,000,000": "$10,000 < $1,000,000"
})

# Replace missing monetary ranges with a default description
jefferson_crime_df['value_range'] = jefferson_crime_df['value_range'].fillna("UNKNOWN RANGE")
jefferson_crime_df['location_category'] = jefferson_crime_df['location_category'].str.replace(" ", "")
location_mapping = {
'RESIDENCE/HOME' : 'RESIDENCE / HOME',
'OTHERRESIDENCE(APARTMENT/CONDO)' : 'APARTMENT / CONDO',
'NON-ATTACHEDRESDGARAGE/SHED/BULD' : 'GARAGE / SHED / OUTBUILDING',
'ATTACHEDRESIDENTIALGARAGE' : 'GARAGE / SHED / OUTBUILDING',
'PARKINGLOT/GARAGE' : 'PARKING LOT / GARAGE',
'PARKING/DROPLOT/GARAGE' : 'PARKING LOT / GARAGE',
'RESTAREA' : 'REST AREA',
'"SPECIALTYSTORE(TV,FUR,ETC)"' : 'SPECIALTY STORE',
'SPECIALTYSTORE' : 'SPECIALTY STORE',
'SCHOOL-ELEMENTARY/SECONDARY' : 'SCHOOL',
'SCHOOLCOLLEGE' : 'COLLEGE',
'SCHOOL-COLLEGE/UNIVERSITY' : 'COLLEGE',
'BANK/SAVINGSANDLOAN' : 'BANK',
'OTHER/UNKOWN' : 'OTHER / UNKNOWN',
'HOTEL/MOTEL/ETC.' : 'HOTEL / MOTEL',
'CAMP/CAMPGROUND' : 'CAMPGROUND',
'CEMETERY/GRAVEYARD' : 'CEMETERY',
'FIELD/WOODS' : 'FIELD / WOODS',
'SERVICE/GASSTATION' : 'GAS STATION',
'CONVENIENCESTORE' : 'CONVENIENCE STORE',
'BAR/NIGHTCLUB' : 'BAR / NIGHTCLUB',
"DRUGSTORE/DOCTOR'SOFFICE/HOSPITAL" :  'DRUGSTORE / DR / HOSPITAL',
'DRUGSTORE/DR`SOFFICE/HOSPITAL' : 'DRUGSTORE / DR / HOSPITAL',
'HIGHWAY/ROAD/ALLEY/STREET/SIDEWALK' : 'ROAD / ALLEY / STREET',
'HIGHWAY/ROAD/ALLEY' : 'ROAD / ALLEY / STREET',
'RESTAURANT' : 'RESTAURANT',
'AUTODEALERSHIP(NEWORUSED)' : 'AUTO DEALERSHIP',
'AUTODEALERSHIPNEW/USED' : 'AUTO DEALERSHIP',
'RENTALSTORAGEFACILITY' : 'STORAGE FACILITY',
'RENTAL/STORAGEFACILITY' : 'STORAGE FACILITY',
'SHOPPINGMALL' : 'MALL',
'MALL/SHOPPINGCENTER' : 'MALL',
'DAYCAREFACILITY' : 'DAYCARE FACILITY',
'PARK/PLAYGROUND' : 'PARK / PLAYGROUND',
'LIQUORSTORE' : 'LIQUOR STORE',
'SHELTER-MISSION/HOMELESS' : 'HOMELESS SHELTER',
'HOMELESSSHELTER/MISSION' : 'HOMELESS SHELTER',
'CHURCH/SYNAGOGUE/TEMPLE/MOSQUE' : 'PLACE OF WORHSIP',
'CHURCH/SYNAGOGUE/TEMPLE' : 'PLACE OF WORSHIP',
'GROCERY/SUPERMARKET' : 'GROCERY STORE',
'DOCK/WHARF/FREIGHT/MODALTERMINAL' : 'DOCK / WHARF / FREIGHT TERMINAL',
'RACETRACK/GAMBLINGFACILITY' : 'RACETRACK / GAMBLING FACILITY',
'GAMBLINGFACILITY/CASINO/RACETRACK' : 'RACETRACK / GAMBLING FACILITY',
'FAIRGROUNDS/STADIUM/ARENA' : 'ARENA / STADIUM / FAIRGROUNDS',
'ARENA/STADIUM/FAIRGROUNDS/COLISEUM' : 'ARENA / STADIUM / FAIRGROUNDS',
'DEPARTMENT/DISCOUNTSTORE' : 'DEPARTMENT STORE',
'COMMUNITYCENTER' : 'COMMUNITY STORE',
'AIR/BUS/TRAINTERMINAL' : 'AIR / BUS / TRAIN TERMINAL',
'ATMSEPARATEFROMBANK' : 'ATM',
'AMUSEMENTPARK' : 'AMUSEMENT PARK',
'ABANDONED/CONDEMNEDSTRUCTURE' : 'CONDEMNED STRUCTURE',
'INDUSTRIALSITE' : 'INDUSTRIAL SITE',
'GOVERNMENT/PUBLICBUILDING' : 'PUBLIC BUILDING',
'COMMERCIAL/OFFICEBUILDING' : 'OFFICE BUILDING'
}

jefferson_crime_df['location_category'] = jefferson_crime_df['location_category'].replace(location_mapping)

# Add columns for the weekday names of the reported and occurred dates
jefferson_crime_df['week_day_reported'] = jefferson_crime_df['date_reported'].dt.day_name()
jefferson_crime_df['week_day_occurred'] = jefferson_crime_df['date_occurred'].dt.day_name()
jefferson_crime_df['zip'] = jefferson_crime_df['zip'].astype('string')
jefferson_crime_df['incident_number'] = jefferson_crime_df['incident_number'].astype('string')
jefferson_crime_df['offense_classification'] = jefferson_crime_df['offense_classification'].astype('category')
jefferson_crime_df['offense_code_name'] = jefferson_crime_df['offense_code_name'].astype('category')
jefferson_crime_df['was_offense_completed'] = jefferson_crime_df['was_offense_completed'].astype('category')
jefferson_crime_df['location_category'] = jefferson_crime_df['location_category'].astype('category')
jefferson_crime_df['week_day_reported'] = jefferson_crime_df['week_day_reported'].astype('category')
jefferson_crime_df['week_day_occurred'] = jefferson_crime_df['week_day_occurred'].astype('category')
jefferson_crime_df['value_range'] = jefferson_crime_df['value_range'].astype('category')

jefferson_crime_df.drop(columns=["offense_code_name"], inplace=True)


# Step 6: Save the combined and cleaned data (optional)
jefferson_crime_df.to_csv('../data/combined_crime_data.csv', index=False)
jefferson_crime_df.dtypes

zip                       string[python]
incident_number           string[python]
date_reported             datetime64[ns]
date_occurred             datetime64[ns]
offense_classification          category
was_offense_completed           category
location_category               category
value_range                     category
week_day_reported               category
week_day_occurred               category
dtype: object